In [ ]:
!pip uninstall numpy -y
!pip install numpy --upgrade --force-reinstall
!pip install tensorflow pandas numpy transformers peft bitsandbytes accelerate datasets
!pip install wordninja
!pip install pyspellchecker
!pip install emoji
!pip install contractions
!pip install joblib
!pip install symspellpy
!pip install tweepy
!pip install gensim

In [ ]:
!pip install --upgrade --force-reinstall pandas scipy scikit-learn gensim

In [ ]:
import kagglehub

path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")

print("Path to dataset files:", path)

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet,words
import seaborn as sns
import matplotlib.pyplot as plt
import gensim
import gensim.downloader as api
from gensim.models import Word2Vec
import tensorflow as tf
import keras
import wordninja
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential,load_model,clone_model
from keras.layers import SimpleRNN,Embedding,Dense,Input,LSTM,LeakyReLU,Activation,Bidirectional,BatchNormalization,LayerNormalization,Dropout,SpatialDropout1D,GlobalAveragePooling1D,MaxPooling1D,Flatten
from keras.preprocessing.sequence import pad_sequences
import re,os
from sklearn.metrics import classification_report,confusion_matrix,f1_score,precision_score,recall_score,roc_auc_score
from spellchecker import SpellChecker
from keras.regularizers import l2
from keras.optimizers import Adam,Adamax,AdamW
from nltk import pos_tag
import emoji
from contractions import fix
from tensorflow.keras.mixed_precision import LossScaleOptimizer
from sklearn.preprocessing import LabelEncoder
import pickle
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
import os
dataset_files = os.listdir(path)
print("Dataset files:", dataset_files)

In [ ]:
import pandas as pd
csv_file = os.path.join(path, "twitter_training.csv")
train_data = pd.read_csv(csv_file,header=None)

train_data.head()


In [ ]:
csv_file = os.path.join(path, "twitter_validation.csv")  # Replace with actual filename
test_data = pd.read_csv(csv_file,header=None)

test_data.head() # Preview the first few rows

In [ ]:
train_data.columns=['id','entity','sentiment','tweet']
test_data.columns=['id','entity','sentiment','tweet']

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.shape

In [ ]:
train_data[train_data['tweet'].isnull()==True]

In [ ]:
train_data['entity'].unique()

In [ ]:
train_data.groupby('sentiment')['tweet'].apply(lambda x: x.isnull().sum())

In [ ]:
train_data=train_data.dropna(subset=['tweet'])
test_data=test_data.dropna(subset=['tweet'])

In [ ]:
def plot_word_cloud(train_data,test_data,col):
  all_tweets = ' '.join(train_data[col].astype(str)) + ' ' + ' '.join(test_data[col].astype(str))
  wordcloud = WordCloud(width=800, height=400, background_color='black', colormap='inferno').generate(all_tweets)
  plt.figure(figsize=(10, 5))
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis('off')
  plt.show()

In [ ]:
plot_word_cloud(train_data,test_data,'tweet')

In [ ]:
def frequently_appearing_words(train_data,test_data,col):
  all_tweets = ' '.join(train_data[col].astype(str)) + ' ' + ' '.join(test_data[col].astype(str))

  # Tokenize and remove stopwords
  stop_words = set(stopwords.words('english'))
  tokens = all_tweets.split()
  filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

  # Get word frequency
  word_counts = Counter(filtered_tokens)

  # Plot most common words
  common_words = word_counts.most_common(10)
  words, counts = zip(*common_words)

  plt.figure(figsize=(10, 5))
  sns.barplot(x=list(words), y=list(counts))
  plt.title('Top 10 Most Frequent Words (Excluding Stopwords)')
  plt.xlabel('Words')
  plt.ylabel('Frequency')
  plt.xticks(rotation=45)
  plt.show()


In [ ]:
frequently_appearing_words(train_data,test_data,'tweet')

## Text preprocessing

In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

In [ ]:
text_emoji_dict = {
    ";D": "😉",  # Winking face with a big grin
    ":D": "😀",  # Grinning face
    ":)": "😊",  # Smiling face
    ":(": "☹️",  # Frowning face
    ":P": "😜",  # Face with stuck-out tongue
    ":p": "😜",  # Face with stuck-out tongue (lowercase)
    ";)": "😉",  # Winking face
    ":O": "😮",  # Face with open mouth
    ":o": "😮",  # Face with open mouth (lowercase)
    ":/": "😕",  # Confused face
    ":\\": "😖",  # Confounded face
    "XD": "😂",  # Laughing face with closed eyes
    ":3": "😺",  # Cat face with smile
    ":|": "😐",  # Neutral face
    ":*": "😘",  # Face blowing a kiss
    ":$": "😳",  # Flushed face
    ":'(": "😢",  # Crying face
    "B)": "😎",  # Smiling face with sunglasses
    "<3": "❤️",  # Heart
    "</3": "💔",  # Broken heart
    ":v": "✌️",  # Victory hand
    "(y)": "👍",  # Thumbs up (commonly used in parenthesis)
    "(n)": "👎",  # Thumbs down (commonly used in parenthesis)
    ":'((": "😭",  # Loudly crying face
    "T_T": "😿",  # Crying face (often seen in anime and manga)
    ";_;": "😿",  # Crying face (sometimes seen in manga style)
    ":|]": "🤖",  # Robot face
    ":x": "❌",  # Cross mark (often representing "no")
    ":o)": "👼",  # Angel face
    ":#": "🤫",  # Shushing face
    ":@": "😡",  # Angry face
    ":))))": "😂",  # Extra happy face
    ":)))": "😂",  # Happy face
    ":]": "🙂",  # Slightly smiling face
    "=D": "😀",  # Big smile
    "=]": "🙂",  # Slightly smiling face
    "O:)": "👼",  # Angel face
    "B|": "😎",  # Cool face with sunglasses
    ":-]": "🙂",  # Smiling face
    ":-(": "☹️",  # Frowning face
    ":-)": "😊",  # Smiling face
    ":-|": "😐",  # Neutral face
    ":-O": "😮",  # Surprised face
    ':S)':'😕', #confused
}
def handle_emoji_text(text):
    # Step 1: Replace text-based emojis with Unicode emojis
    for text_emoji, unicode_emoji in text_emoji_dict.items():
        text = text.replace(text_emoji, unicode_emoji)

    # Step 2: Convert Unicode emojis to text descriptions using emoji.demojize
    text = emoji.demojize(text)

    return text



## handling emojis and handling html characters

In [ ]:
def handle_unk(text):
    # Replace both "<unk>" and "< unk >" with an empty string
    text = re.sub(r'<\s*unk\s*>', '', text)
    return text

In [ ]:
import spacy
import re

# Load SpaCy's NER model
nlp = spacy.load("en_core_web_sm", disable=['parser', 'tagger'])
def remove_company_names(text):
    """Remove company names (ORG entities) from text."""
    list_of_companies=['Borderlands', 'CallOfDutyBlackopsColdWar', 'Amazon', 'Overwatch',
       'Xbox(Xseries)', 'NBA2K', 'Dota2', 'PlayStation5(PS5)',
       'WorldOfCraft', 'CS-GO', 'Google', 'AssassinsCreed', 'ApexLegends',
       'LeagueOfLegends', 'Fortnite', 'Microsoft', 'Hearthstone',
       'Battlefield', 'PlayerUnknownsBattlegrounds(PUBG)', 'Verizon',
       'HomeDepot', 'FIFA', 'RedDeadRedemption(RDR)', 'CallOfDuty',
       'TomClancysRainbowSix', 'Facebook', 'GrandTheftAuto(GTA)',
       'MaddenNFL', 'johnson&johnson', 'Cyberpunk2077',
       'TomClancysGhostRecon', 'Nvidia','Netflix','IPhone','MAC','Windows','Twitter',
                       'YouTube','GeForce','ChatGPT','Tesla','Ryzen','AMD','ASUS','HDMI',
                       'Apple','PS5','PS4','PS','PS3','PS2']
    list_of_companies_lower_case=[i.lower() for i in list_of_companies]

    """Remove company names from text using both NER and a predefined list."""
    doc = nlp(text)

    # Step 1: Remove company names detected by NER
    filtered_tokens = [token.text for token in doc if token.ent_type_ != "ORG"]
    cleaned_text = " ".join(filtered_tokens)

    # Step 2: Remove lowercase company names from predefined list
    pattern = r'\b(?:' + '|'.join(re.escape(company) for company in list_of_companies) + r')\b'
    cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE).strip()

    return cleaned_text

# Example Usage
text = "google and microsoft are leading AI research. Windows windows WINDOWS Google apple just launched a new iphone. PS5 netflix is popular."
cleaned_text = remove_company_names(text)
print(cleaned_text)



In [ ]:
!wget https://raw.githubusercontent.com/mammothb/symspellpy/master/symspellpy/frequency_dictionary_en_82_765.txt

## Handling chat texts

In [ ]:
chat_dict = {
    "n": "and", "s": "yes", "2 ya": "to you", "to ya": "to you", "4 ya": "for you",
    "c ya": "see you", "for ya": "for you", "2day": "today", "bro": "brother",
    "how ya": "how you", "watcha": "what are", "shoulda": "should have",
    "watcha doin": "what are you doing", "sucks": "suck", "sucked": "suck",
    "sucking": "suck", "kinda": "kind of", "sorta": "sort of", "wrd": "what are you doing",
    "lamfao": "laughing my fucking ass off", "wht": "what", "yas": "yes",
    "abt": "about", "luk": "look", "grl": "girl", "cum": "come", "boi": "boy",
    "bday": "birthday", "b day": "birthday", "asap": "as soon as possible",
    "nvm": "never mind", "dm": "direct message", "d": "the", "grt": "great",
    "fck": "fuck", "lol": "laugh out loud", "omg": "oh my god", "brb": "be right back",
    "ttyl": "talk to you later", "bff": "best friend forever", "wtf": "what the fuck",
    "idk": "I don't know", "plz": "please", "thx": "thanks", "gr8": "great",
    "w8": "wait", "h8": "hate", "u2": "you too", "l8r": "later",
    "imho": "in my humble opinion", "jk": "just kidding", "smh": "shaking my head",
    "fyi": "for your information", "xoxo": "hugs and kisses", "cya": "see you",
    "np": "no problem", "4u": "for you", "yw": "you're welcome", "tbh": "to be honest",
    "gtg": "got to go", "k": "okay", "yo": "hello", "wbu": "what about you",
    "wym": "what you mean", "fml": "fuck my life", "wyd": "what are you doing",
    "boo": "friend", "aww": "affection", "omfgz": "oh my fucking god", "msg": "message",
    "imo": "in my opinion", "paw": "parents are watching", "pitr": "parent in the room",
    "pbb": "parent behind back", "poms": "parent over my shoulder", "kpc": "keeping parents clueless",
    "mrw": "my reaction when", "ily": "I love you", "adih": "another day in hell",
    "zzz": "sleeping, bored, tired", "wywh": "wish you were here", "pov": "point of view",
    "gl": "good luck", "hbd": "happy birthday", "bf": "boyfriend", "gf": "girlfriend",
    "tysm": "thank you so much", "lmao": "laughing my ass off",
    "rn": "right now", "m": "am","im":"i am"
}

In [ ]:
def replace_chat_words(text):
    if isinstance(text, str):
        words = text.split()
        replaced_words = [chat_dict.get(word.lower(), word) for word in words]
        joined_words = ' '.join(replaced_words)

        joined_words = re.sub(r"\bi m\b", "i am", joined_words)
        joined_words = re.sub(r"\bto you\b", "to you", joined_words)
        joined_words = re.sub(r"\b4 you\b", "for you", joined_words)
        joined_words = re.sub(r"\b4 ya\b", "for you", joined_words)
        joined_words = re.sub(r"\b2 you\b", "to you", joined_words)
        joined_words = re.sub(r"\b2 ya\b", "to you", joined_words)
        joined_words = re.sub(r"\bc ya\b", "see you", joined_words)
        joined_words = re.sub(r"\bsee you\b", "see you", joined_words)

        return joined_words.strip()

    return text


In [ ]:
emotion_dict = {
    "surprise": [
        "ah", "ahhh", "ahhhh", "ahhhhh", "oh", "ohhh", "ohhhh", "ohhhhh", "whoa", "whoaa", "whoaaa", "wow", "woah", "woaaa",
        "whoops", "huh", "no way", "seriously", "really?"
    ],
    "disgust": [
        "ew", "ewww", "ewwww", "ewwwww", "yuck", "yucky", "gross", "nasty", "bleh", "ugh", "yikes", "blech", "barf", "vomit",
        "gag", "puke", "sick", "shudder"
    ],
    "fear": [
        "ahh", "eek", "ohno", "ohnoooo", "noooo", "nooooo", "scared", "frightened", "help", "panic", "terrified", "shocked",
        "spooked", "freaked", "yikes", "shudder", "screech", "screaming", "gasp"
    ],
    "happiness": [
        "yay", "wohoo", "whoohoo",'hah' "yayyy", "yass", "yasss", "hehe", "haha", "hooray", "yippee", "woohoo", "whee", "yup",
        "whoo", "wooo", "yayyy", "yasss", "yayyy", "laughter", "chuckle", "giggle", "snicker",'woa'
    ],
    "sadness": [
        "sob", "cry", "tears", "sigh", "sad", "waah", "waaaa", "boohoo", "sniffle", "wail", "sniff", "whimper", "weep",
        "sniffing", "blubber", "despair", "gasp"
    ],
    "excitement": [
        "woo", "wooo",'woo', "wooohoo", "yayyyy",'yesss', "yaay", "yaaaay", "excited", "yippee", "whoop", "wooohooo", "hype", "yeah",
        "wahoo", "booyah", "let's go", "jump", "bounce", "get hyped", "pumped", "woot", "yeeeah",'ha'
    ],
    "anger": [
        "grr", "grrrr", "arrgh", "ugh", "ughhh", "arghhh", "noooo", "damn", "dammit", "shit", "pissed", "furious", "irritated",
        "annoyed", "angry", "rage", "livid", "fuming", "growl", "sigh", "huff", "groan", "grumble"
    ],
    "love": [
        "aw", "aww", "yay", "sweet", "adorable", "cute", "love", "heart", "xoxo", "hug", "kiss", "muah", "swoon", "feels",
        "heartfelt", "romantic", "my love", "forever", "mine", "baby", "babe"
    ],
    "boredom": [
        "meh", "blah", "sigh", "yawn", "bored", "uninterested", "tired", "apathetic"
    ],
    "confusion": [
        "huh", "what?", "eh", "umm", "uh", "hmmm", "no idea", "IDK", "beats me", "confused", "perplexed", "puzzled",
        "wth", "wtf", "shaking head"
    ],
    "shame": [
        "oops", "whoops", "sorry", "embarrassed", "blushing", "awkward", "guilt", "my bad", "facepalm", "red-faced",
        "cringe", "yikes", "blush"
    ],
    "pride": [
        "yeah", "whew", "whoo", "hell yeah", "that's right", "woohoo", "yes", "we did it", "booyah", "gotcha", "rock on",
        "congratulations", "you go", "victory", "champion"
    ],
    "relief": [
        "phew", "whew"
    ]
}

# Normalize repeated characters for emotional expressions
def normalize_repeated_characters(text):
    # Normalize repeated characters for words like "ahhh", "ohhh", "ewww"
    text = re.sub(r'(ah|oh|ew|woo|yay|grr)\1{1,}', r'\1', text)
    return text

def replace_emotions(text):
    # Step 1: Normalize repeated characters first
    text = normalize_repeated_characters(text)

    # Step 2: Loop through the emotion_dict and replace expressions with emotions
    for emotion, words in emotion_dict.items():
        for word in words:
            # Use regex to replace word (case insensitive)
            text = re.sub(r'\b' + re.escape(word) + r'\b', emotion, text, flags=re.IGNORECASE)

    return text


In [ ]:
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell.load_dictionary("frequency_dictionary_en_82_765.txt", term_index=0, count_index=1)


def correct_sentence(sentence):
    words = sentence.split()
    corrected_words = [
        sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)[0].term if sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2) else word
        for word in words
    ]
    return " ".join(corrected_words)


In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Function for batch lemmatization
def batch_lemmatization(texts):
    """Lemmatize a list of texts."""
    docs = list(nlp.pipe(texts, batch_size=100))
    return [" ".join([token.lemma_ for token in doc]) for doc in docs]


In [ ]:
import re

def fix_contractions(text):
    text = re.sub(r'[ ]+', ' ', text).strip()

    text = re.sub(r"\b(\w+)\s*'\s*t\b", r"\1't", text)  # Fix "didn ' t" -> "didn't"
    text = re.sub(r"\b(\w+)\s*'\s*s\b", r"\1's", text)    # Fix "it ' s" -> "it's"
    text = re.sub(r"\b(\w+)\s*'\s*m\b", r"\1'm", text)    # Fix "I ' m" -> "I'm"
    text = re.sub(r"\b(\w+)\s*'\s*ve\b", r"\1've", text)  # Fix "I ' ve" -> "I've"
    text = re.sub(r"\b(\w+)\s*'\s*d\b", r"\1'd", text)    # Fix "he ' d" -> "he'd"
    text = re.sub(r"\b(\w+)\s*'\s*re\b", r"\1're", text)  # Fix "you ' re" -> "you're"
    text = re.sub(r"\b(\w+)\s*'\s*ll\b", r"\1'll", text)  # Fix "she ' ll" -> "she'll"

    return text

In [ ]:
import html
def clean_encoding(text):

    text = text.replace('ï¿½', "'")
    #print(f"After manual encoding fix: {text}")

    # Decode HTML entities (e.g., &lt; -> <, &quot; -> ")
    text = html.unescape(text)
    #print(f"After unescaping HTML entities: {text}")

    return text

In [ ]:
def normalize_quotes(text):
    text = re.sub(r"[‘’]", "'", text)
    text = re.sub(r"[“”]", '"', text)
    return text

In [ ]:
def normalize_elongated(text):
    text = text.lower()

    # Reduce repeated characters:
    # - If the whole word is made of one repeated letter, keep only one
    # - Otherwise, reduce repeated characters to a maximum of 2
    words = text.split()
    processed_words = [
        re.sub(r'(.)\1+', r'\1', word) if len(set(word)) == 1  # If only one unique letter, keep single letter
        else re.sub(r'(.)\1{2,}', r'\1\1', word)  # Otherwise, reduce to max 2 occurrences
        for word in words
    ]
    text=' '.join(processed_words)

    return text


In [ ]:
from nltk.tokenize import TweetTokenizer


donot_remove = {'am','not', 'have', 'has', 'must', 'should', 'had', 'might','never','was'}
stop_words = set(stopwords.words('english')) - donot_remove

tokenizer = TweetTokenizer(preserve_case=False, strip_handles=False, reduce_len=True)

In [ ]:
def clean_text(text):
  if isinstance(text,str):
    text = clean_encoding(text)
    text = emoji.demojize(text)
    text = handle_emoji_text(text)


    text = text.lower()

    text = normalize_quotes(text)
    text = fix_contractions(text)
    text = fix(text)


    text = re.sub(r'http\S+|www\S+|[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/\S*)?', "", text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#(\w+)',r'\1',text)

    text = remove_company_names(text)

    text = normalize_elongated(text)

    text = replace_chat_words(text)
    text = replace_emotions(text)


    text = re.sub(r"[^a-zA-Z\s]",'',text)
    text = re.sub(r"\s+", " ", text).strip()
    text = correct_sentence(text)
    text = batch_lemmatization([text])[0]
    tokens = tokenizer.tokenize(text)
    tokens=[word for word in tokens if word not in stop_words]


    return tokens
  return text


In [ ]:
train_data['processed_text']=train_data['tweet'].apply(clean_text)
test_data['processed_text']=test_data['tweet'].apply(clean_text)

In [ ]:
train_data['processed_text']

In [ ]:
train_data

In [ ]:
train=train_data.copy()
test=test_data.copy()

In [ ]:
train['sentiment'].value_counts()

In [ ]:
sns.countplot(x='sentiment',data=train)
plt.show()

In [ ]:
sns.countplot(x='sentiment',data=test)
plt.show()

In [ ]:
# remove irrelevant

train=train[train['sentiment']!='Irrelevant']
train['sentiment'].value_counts()

In [ ]:
train.shape

In [ ]:
test=test[test['sentiment']!='Irrelevant']
test['sentiment'].value_counts()

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
le=LabelEncoder()
train_data['sentiment']=le.fit_transform(train_data['sentiment'])
test_data['sentiment']=le.transform(test_data['sentiment'])

In [ ]:
path='/content/MyDrive/MyDrive/Twitter sentiment analysis processed data/label_encoder.pkl'
with open(path, "wb") as f:
    pickle.dump(le, f)


In [ ]:
# Load the LabelEncoder from the pickle file
with open(path, "rb") as f:
    le = pickle.load(f)

In [ ]:
train=pd.read_csv('/content/MyDrive/MyDrive/Twitter sentiment analysis processed data/train_tweets.csv')
test=pd.read_csv('/content/MyDrive/MyDrive/Twitter sentiment analysis processed data/test_tweets.csv')
train.head()

In [ ]:
train['sentiment'].value_counts()

In [ ]:
test['sentiment'].value_counts()

In [ ]:
import ast
train['processed_text'] = train['processed_text'].apply(lambda x: ast.literal_eval(x))
test['processed_text'] = test['processed_text'].apply(lambda x: ast.literal_eval(x))

In [ ]:
csv_path = "/content/drive/My Drive/Twitter_Sentiment_Analysis/processed_tweets"
os.makedirs(csv_path, exist_ok=True)

train['processed_text'] = train['processed_text'].apply(lambda x: ','.join(x))
test['processed_text'] = test['processed_text'].apply(lambda x: ','.join(x))


train_data.to_csv(os.path.join(csv_path, "train_tweets.csv"), index=False)
test_data.to_csv(os.path.join(csv_path, "test_tweets.csv"), index=False)

import pickle

with open("text_preprocessing.pkl", "wb") as f:
    pickle.dump(clean_text, f)

print("Function saved successfully!")


In [ ]:
train

In [ ]:
train_processed_text = ' '.join(train['processed_text'].apply(lambda x: ' '.join(x)))
test_processed_text = ' '.join(test['processed_text'].apply(lambda x: ' '.join(x)))


combined_processed_text = train_processed_text + " " + test_processed_text


wordcloud = WordCloud(width=800, height=400,background_color='black', colormap='inferno').generate(combined_processed_text)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Processed Text')
plt.show()

In [ ]:
frequently_appearing_words(train,test,'processed_text')

## sentiment analysis using GloVe

In [ ]:
!wget https://nlp.stanford.edu/data/glove.twitter.27B.zip

In [ ]:
#unzip
import zipfile
zip_ref=zipfile.ZipFile('/content/glove.twitter.27B.zip')
zip_ref.extractall()
zip_ref.close()

In [ ]:
train_texts = train['processed_text'].fillna("")
test_texts = test['processed_text'].fillna("")
train_labels = train['sentiment'].values
test_labels = test['sentiment'].values

In [ ]:
train_texts

In [ ]:
train_texts.shape,test_texts.shape

## Finding the optimal num_words

In [ ]:
from collections import Counter
word_counts = Counter(word for tweet in train_texts for word in tweet)

sorted_counts=sorted(word_counts.values(),reverse=True)
sorted_counts[:10]

In [ ]:
print(word_counts.most_common(50))  # Show top 20 words with counts


In [ ]:
#Find the total number of words that covers 90% of occurences
total_words=sum(sorted_counts)
running_sum=0
num_words=0
for count in sorted_counts:
  running_sum+=count
  num_words+=1
  if running_sum/total_words >= 0.90:
    break
print(f'Optimal num_words for 90% coverage: {num_words}')


In [ ]:
import numpy as np
np.bincount(train_labels)  # Count occurrences of each class

In [ ]:
train_labels

In [ ]:
train_texts=train['processed_text']
test_texts=test['processed_text']

In [ ]:
train_texts,test_texts

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


num_words = 20000  # Maximum allowed vocabulary size
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = num_words,oov_token=oov_token)
tokenizer.fit_on_texts(train_texts)

# Convert text to sequences
x_train = tokenizer.texts_to_sequences(train_texts)
x_test = tokenizer.texts_to_sequences(test_texts)

vocab_size = len(tokenizer.word_index) +1


'''x_train = [[min(w, vocab_size - 1) for w in seq] for seq in x_train]
x_test = [[min(w, vocab_size - 1) for w in seq] for seq in x_test]'''

oov_token_index = tokenizer.word_index.get("<OOV>", 1)

# Replace out-of-bounds indices with the OOV token index
#x_train = [[w if w < vocab_size else oov_token_index for w in seq] for seq in x_train]
#x_test = [[w if w < vocab_size else oov_token_index for w in seq] for seq in x_test]


#x_train = [[w if w < vocab_size else 1 for w in seq] for seq in x_train]
#x_test = [[w if w < vocab_size else 1 for w in seq] for seq in x_test]

x_train = [[min(idx, vocab_size - 1) for idx in seq] for seq in x_train]
x_test = [[min(idx, vocab_size - 1) for idx in seq] for seq in x_test]

# Compute max sequence length (95th percentile)
max_length = int(np.percentile([len(seq) for seq in x_train], 95))

# Pad sequences
x_train = pad_sequences(x_train, maxlen=max_length, padding="post",truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding="post",truncating='post')

word_index = tokenizer.word_index



# Debugging info
print("Max word index:", max(word_index.values()))
print(f"Unique words in training set: {len(word_index)}")
print(f"Max sequence length: {max_length}")



In [ ]:
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [ ]:
x_train

In [ ]:
max_length

In [ ]:
oov_count = sum(1 for seq in x_train for w in seq if w == 1)
print("OOV Token Count in Training Data:", oov_count)

In [ ]:
type(x_train),type(x_test)

In [ ]:
len(x_train)

In [ ]:
y_train = train['sentiment'].values
y_test = test['sentiment'].values

In [ ]:
y_train,y_test

In [ ]:
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

y_train.shape,y_test.shape

In [ ]:
import numpy as np

# Save all datasets in a single .npz file
np.savez("train_test_split.npz", x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

In [ ]:
y_train.shape,y_test.shape

In [ ]:
x_train.shape,train_labels.shape

In [ ]:
x_train[:1]

In [ ]:
x_train.shape,x_test.shape

In [ ]:
x_train[:10]

In [ ]:
x_train.shape[1]

In [ ]:
import numpy as np

embedding_dim = 100
embedding_matrix = np.random.uniform(-0.05, 0.05, (vocab_size, embedding_dim))

word_found = 0
with open("glove.twitter.27B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        if word in tokenizer.word_index and tokenizer.word_index[word] < vocab_size - 1:
            embedding_matrix[tokenizer.word_index[word]] = np.asarray(values[1:], dtype='float32')
            word_found += 1

# Compute mean of known embeddings only if there are known words
known_embeddings = embedding_matrix[np.any(embedding_matrix != 0, axis=1)]
if word_found > 0:
    mean_embedding = np.mean(known_embeddings, axis=0)
else:
    mean_embedding = np.random.uniform(-0.05, 0.05, embedding_dim)

embedding_matrix[np.all(embedding_matrix == 0, axis=1)] = mean_embedding

print(f"Embedding matrix shape: {embedding_matrix.shape}")
print(f"Words found in GloVe: {word_found}")



In [ ]:
#normalize the embedding
from sklearn.preprocessing import normalize
embedding_matrix = normalize(embedding_matrix, axis=1)

In [ ]:
np.save('embedding_matrix.npy', embedding_matrix)

In [ ]:
embedding_matrix[:1]

## Training without using glove embedding

In [ ]:
print(tf.keras.mixed_precision.global_policy())

In [ ]:
max_length

In [ ]:
num_words

In [ ]:
vocab_size

In [ ]:
embedding_dim,max_length

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  # Plot the heatmap
  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels, cbar=False)

  plt.title('Confusion Matrix', fontsize=16)
  plt.xlabel('Predicted Labels', fontsize=12)
  plt.ylabel('True Labels', fontsize=12)


  plt.show()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.regularizers import l2
import tensorflow as tf


dep_model0= Sequential()

# Embedding Layer (Pretrained GloVe, Non-Trainable)
dep_model0.add(Embedding(input_dim=vocab_size,
                    output_dim=100,
                    input_length=max_length,
                    weights=[embedding_matrix],
                    trainable=False))


dep_model0.add(SpatialDropout1D(0.3))

dep_model0.add(LSTM(32,
               dropout=0.3,
               recurrent_dropout=0.3,
               return_sequences=False,
               kernel_regularizer=l2(0.001)))

dep_model0.add(BatchNormalization())
dep_model0.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
dep_model0.add(Dropout(0.4))

dep_model0.add(Dense(3, activation='softmax'))


dep_model0.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-6)
]


dep_model0.summary()

In [ ]:
history0 = dep_model0.fit(x_train, y_train,
                        epochs=20,
                        batch_size=32,
                        validation_data=(x_test, y_test),
                        callbacks=callbacks)

In [ ]:
dep_model0.evaluate(x_train,y_train),dep_model0.evaluate(x_test,y_test)

In [ ]:
y_pred=dep_model0.predict(x_test)
y_pred

In [ ]:
y_pred_classes = np.argmax(y_pred, axis=1)
y_pred_classes[:10]

In [ ]:
y_test[:10]

In [ ]:
y_test_classes = np.argmax(y_test, axis=1)
y_test_classes[:10]

In [ ]:
cm = confusion_matrix(y_test_classes, y_pred_classes)

class_labels = ['Negative', 'Neutral','Positive']
plot_confusion_matrix(cm,class_labels)

In [ ]:
print(classification_report(y_test_classes,y_pred_classes))

In [ ]:
print(f'Precision: {precision_score(y_test_classes,y_pred_classes,average="weighted")}')
print(f'Recall: {recall_score(y_test_classes,y_pred_classes,average="weighted")}')
print(f'F1 Score: {f1_score(y_test_classes,y_pred_classes,average="weighted")}')

## Fine-tuning using LSTM model

In [ ]:
from keras.regularizers import l2

#opt = LossScaleOptimizer(tf.keras.optimizers.AdamW(learning_rate=0.001,weight_decay=0.0001))
opt=tf.keras.optimizers.AdamW(learning_rate=0.001,weight_decay=0.0001)
embedding_dim = 100

embedding_layer=Embedding(input_dim=vocab_size,
                         output_dim=embedding_dim,
                         input_length=max_length,
                         weights=[embedding_matrix],
                         trainable=True
                         )

dep_model = Sequential()
dep_model.add(embedding_layer)

dep_model.add(SpatialDropout1D(0.2))
dep_model.add(LSTM(64, dropout=0.3, return_sequences=True,recurrent_dropout=0.3,kernel_regularizer=l2(0.005)))
dep_model.add(LayerNormalization())
dep_model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.3,kernel_regularizer=l2(0.001)))
dep_model.add(BatchNormalization())
dep_model.add(Dense(64, activation='relu',kernel_regularizer=l2(0.001)))
dep_model.add(Dropout(0.3)),
dep_model.add(Dense(3, activation='softmax'))

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-6)
]

dep_model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

dep_model.summary()

In [ ]:
x_train.shape,y_train.shape

In [ ]:
print(x_train.shape)
print(x_test.shape)


In [ ]:
history = dep_model.fit(x_train, y_train,
                        epochs=10,
                        batch_size=32,
                        validation_data=(x_test, y_test),
                        callbacks=callbacks)


In [ ]:
dep_model.evaluate(x_train,y_train),dep_model.evaluate(x_test,y_test)

In [ ]:
y_pred=dep_model.predict(x_test)
y_pred

In [ ]:
y_pred_classes = np.argmax(y_pred, axis=1)
y_pred_classes[:10]

In [ ]:
y_test[:10]

In [ ]:
y_test_classes = np.argmax(y_test, axis=1)  # Convert one-hot to class labels

In [ ]:
# Print the classes that were encoded
print("Classes: ", le.classes_)

# To map the predicted labels or encoded labels back to original sentiment labels
predicted_sentiment = le.inverse_transform(y_pred_classes)
print(predicted_sentiment[:10])  # Show the first 10 predictions as an example


In [ ]:
cm = confusion_matrix(y_test_classes, y_pred_classes)

class_labels = ['Negative', 'Neutral','Positive']
plot_confusion_matrix(cm,class_labels)

In [ ]:
print(classification_report(y_test_classes,y_pred_classes))

In [ ]:
print(f'Precision: {precision_score(y_test_classes,y_pred_classes,average="weighted")}')
print(f'Recall: {recall_score(y_test_classes,y_pred_classes,average="weighted")}')
print(f'F1 Score: {f1_score(y_test_classes,y_pred_classes,average="weighted")}')

In [ ]:
dep_model.save("dep_model.h5")

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model("dep_model.h5")

## Using bidirectional LSTM

In [ ]:
loaded_embedding_matrix = np.load('/content/MyDrive/MyDrive/Twitter sentiment analysis processed data/embedding_matrix.npy')
loaded_embedding_matrix

In [ ]:
import pickle

# Load the tokenizer from the pickle file
with open("/content/MyDrive/MyDrive/Twitter sentiment analysis processed data/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [ ]:
dep_model3 = Sequential()

opt=tf.keras.optimizers.AdamW(learning_rate=0.001,weight_decay=0.0001)

embedding_layer1 = Embedding(input_dim=vocab_size,
                             output_dim=100,
                             input_length=26,
                             weights=[loaded_embedding_matrix],
                             trainable=False)

dep_model3.add(embedding_layer1)

dep_model3.add(Bidirectional(LSTM(64, dropout=0.3, return_sequences=False, recurrent_dropout=0.3, kernel_regularizer=l2(0.001))))
dep_model3.add(BatchNormalization())
dep_model3.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
dep_model3.add(BatchNormalization())
dep_model3.add(Dropout(0.3))
dep_model3.add(Dense(3, activation='softmax'))

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-6)
]

dep_model3.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
dep_model3.summary()

In [ ]:
y_train

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
history3 = dep_model3.fit(x_train, y_train,
                        epochs=20,
                        batch_size=32,
                        validation_data=(x_test, y_test),
                        callbacks=callbacks)

In [ ]:
dep_model3.evaluate(x_train,y_train),dep_model3.evaluate(x_test,y_test)

In [ ]:
def plot_loss_acc_curves(history):
  plt.plot(history.history['loss'], label='Training Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  plt.figure()
  plt.plot(history.history['accuracy'], label='Training Accuracy')
  plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

In [ ]:
plot_loss_acc_curves(history3)

In [ ]:
y_test[:5]

In [ ]:
y_test_classes=np.argmax(y_test,axis=1)
y_test_classes[:5]

In [ ]:
y_pred=dep_model3.predict(x_test)
y_pred

In [ ]:
y_pred_classes=np.argmax(y_pred,axis=1)
y_pred_classes[:10]

In [ ]:
import pickle

# Load the tokenizer from the pickle file
with open("/content/MyDrive/MyDrive/Twitter sentiment analysis processed data/label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

In [ ]:
print("Classes: ", le.classes_)

In [ ]:
cm=confusion_matrix(y_test_classes,y_pred_classes)
class_labels=le.classes_
plot_confusion_matrix(cm,class_labels)

In [ ]:
print(classification_report(y_test_classes,y_pred_classes))

In [ ]:
data = np.load("/content/MyDrive/MyDrive/Twitter sentiment analysis processed data/train_test_split.npz")


x_train = data["x_train"]
x_test = data["x_test"]
y_train = data["y_train"]
y_test = data["y_test"]
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
type(x_train)

## Adding Global Average pooling 1D after Bidirectional LSTM

In [ ]:
dep_model4=Sequential([
    Embedding(input_dim=vocab_size,
                             output_dim=100,
                             input_length=26,
                             weights=[loaded_embedding_matrix],
                             trainable=True),
    Bidirectional(LSTM(64, dropout=0.3, return_sequences=True, recurrent_dropout=0.3, kernel_regularizer=l2(0.001))),
    GlobalAveragePooling1D(),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(3, activation='softmax')
])
opt=tf.keras.optimizers.AdamW(learning_rate=0.001,weight_decay=0.0001)
dep_model4.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])
dep_model4.summary()

In [ ]:
callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True),
           tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-6)]
history4=dep_model4.fit(x_train,y_train,
               epochs=20,
               batch_size=32,
               validation_data=(x_test,y_test),
               callbacks=callbacks)

In [ ]:
dep_model4.save("dep_model4.h5")

In [ ]:
from tensorflow.keras.models import load_model

# Load the model from HDF5 file
model = load_model("dep_model4.h5")


In [ ]:
model.evaluate(x_train,y_train),model.evaluate(x_test,y_test)

In [ ]:
y_pred=model.predict(x_test)
y_pred

In [ ]:
y_pred_classes=np.argmax(y_pred,axis=1)
y_pred_classes[:10]

In [ ]:
y_test_classes=np.argmax(y_test,axis=1)
y_test_classes[:10]

In [ ]:
cm=confusion_matrix(y_test_classes,y_pred_classes)
class_labels=le.classes_
plot_confusion_matrix(cm,class_labels)

In [ ]:
print(classification_report(y_test_classes,y_pred_classes))

In [ ]:
import pickle

# Load preprocessing objects (e.g., tokenizer)
path='/content/MyDrive/MyDrive/Twitter sentiment analysis processed data/text_preprocessing.pkl'
with open(path, "rb") as f:
    preprocessing_pipeline = pickle.load(f)


In [ ]:
print(type(preprocessing_pipeline))  # Check the data type
print(preprocessing_pipeline)  # Print details

In [ ]:
import pickle

tokenizer_path = "/content/MyDrive/MyDrive/Twitter sentiment analysis processed data/tokenizer.pkl"

with open(tokenizer_path, "rb") as f:
    tokenizer = pickle.load(f)

print(type(tokenizer))


In [ ]:
from nltk.tokenize import TweetTokenizer


donot_remove = {'am','not', 'have', 'has', 'must', 'should', 'had', 'might','never','was'}
stop_words = set(stopwords.words('english')) - donot_remove


tokenizer = TweetTokenizer(preserve_case=False, strip_handles=False, reduce_len=True)

In [ ]:
num_words = 20000
oov_token = "<OOV>"

tokenizer1 = Tokenizer(num_words = num_words,oov_token=oov_token)
tokenizer1.fit_on_texts(train_texts)

In [ ]:
def predict_sentiment(custom_tweet, preprocessing_pipeline, tokenizer, model, max_length=26):
    """
    Predicts the sentiment of a given tweet.

    Parameters:
    - custom_tweet (str): The input tweet to analyze.
    - preprocessing_pipeline (function): A function for text preprocessing.
    - tokenizer (Tokenizer): The trained tokenizer for text-to-sequence conversion.
    - model (tf.keras.Model): The trained sentiment analysis model.
    - max_length (int): The maximum sequence length for padding (default: 26).

    Returns:
    - str: The predicted sentiment ("Negative", "Neutral", or "Positive").
    """
    # Preprocess the tweet
    cleaned_tweet = preprocessing_pipeline(custom_tweet)

    # Convert text to sequence using the loaded tokenizer
    custom_tweet_seq = tokenizer.texts_to_sequences([cleaned_tweet])

    # Pad the sequence
    custom_tweet_padded = tf.keras.preprocessing.sequence.pad_sequences(custom_tweet_seq, maxlen=max_length,padding='post',truncating='post')

    # Make Prediction
    prediction = model.predict(custom_tweet_padded)

    # Convert Prediction to Sentiment Label
    sentiment_labels = ["Negative", "Neutral", "Positive"]
    predicted_class = sentiment_labels[np.argmax(prediction)]

    # Print results
    print("Cleaned Tweet:", cleaned_tweet)
    print("Tokenized Sequence:", custom_tweet_seq)
    print("Padded Sequence:", custom_tweet_padded)
    print("Prediction Probabilities:", prediction)
    print("Predicted Sentiment:", predicted_class)

    return predicted_class



In [ ]:
tweet = "I love this product!"
predicted_sentiment = predict_sentiment(tweet, preprocessing_pipeline, tokenizer1, model)
print(f"Final Predicted Sentiment: {predicted_sentiment}")

In [ ]:
print(tokenizer1.word_index)  # Check if words are indexed correctly


In [ ]:
train[train['tweet'].str.contains('laugh', case=False, na=False)]['sentiment'].value_counts()

In [ ]:
train[(train['tweet'].str.contains('laugh', case=False, na=False)) & (train['sentiment'] == 0)].head(10)


In [ ]:
import numpy as np

word = "laugh"
if word in word_index:
    word_vec = loaded_embedding_matrix[word_index[word]]
    print(f"Embedding for '{word}':", word_vec)
else:
    print(f"'{word}' not found in vocabulary!")


In [ ]:
tweets=["I didn't like it ",
        "I was a boring movie",
        'What a nice girl',
        'She is a positive girl',
        'fucked up',
        'you suck',
        'I got a great feeling about this',
        'I am beautiful',
        'She is a woman',
        'I am a bastard lol!',
        'I am laughing',
        "I love to laugh, it's the best feeling!",
        'He is showing off his abs']
for tweet in tweets:
  predicted_sentiment = predict_sentiment(tweet, preprocessing_pipeline, tokenizer1, model)
  print(f"Final Predicted Sentiment: {predicted_sentiment}")

In [ ]:
import collections

label_counts = collections.Counter(y_train.argmax(axis=1))
print(label_counts)


## Adding MaxPool1D instead of Global average pooling

In [ ]:
x_train = np.array(x_train, dtype=np.int32)
x_test=np.array(x_test,dtype=np.int32)
y_train = np.array(y_train, dtype=np.float32)
y_test=np.array(y_test,dtype=np.float32)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
dep_model5=Sequential([
    Embedding(input_dim=vocab_size,
                             output_dim=100,
                             input_length=26,
                             weights=[loaded_embedding_matrix],
                             trainable=True),
    Bidirectional(LSTM(64, dropout=0.3, return_sequences=True, recurrent_dropout=0.3, kernel_regularizer=l2(0.001))),
    MaxPooling1D(pool_size=2),
    Flatten(),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(3, activation='softmax')
])
opt=tf.keras.optimizers.AdamW(learning_rate=0.001,weight_decay=0.0001)
dep_model5.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])
dep_model5.summary()

In [ ]:
callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True),
           tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-6)]
history5=dep_model5.fit(x_train,y_train,
                        epochs=20,
                        validation_data=(x_test,y_test),
                        batch_size=32,
                        callbacks=callbacks)

In [ ]:
plot_loss_acc_curves(history5)

In [ ]:
dep_model5.evaluate(x_train,y_train),dep_model5.evaluate(x_test,y_test)

In [ ]:
y_test_classes=np.argmax(y_test,axis=1)
y_pred_classes=np.argmax(dep_model5.predict(x_test),axis=1)
cm=confusion_matrix(y_test_classes,y_pred_classes)
class_labels=le.classes_
plot_confusion_matrix(cm,class_labels)

In [ ]:
tweets=["I didn't like it ",
        "I was a boring movie",
        'What a nice girl',
        'She is a positive girl',
        'fucked up',
        'you suck',
        'I got a great feeling about this',
        'I am beautiful',
        'She is a woman',
        'I am a bastard lol!',
        'I am laughing',
        'You are truely blessed',
        "I love to laugh, it's the best feeling!",
        'He is showing off his abs']
for tweet in tweets:
  predicted_sentiment = predict_sentiment(tweet, preprocessing_pipeline, tokenizer1, dep_model5)
  print(f"Final Predicted Sentiment: {predicted_sentiment}")

## Stacking two Bilstm layers

In [ ]:
dep_model6=Sequential([
    Embedding(input_dim=vocab_size,
                             output_dim=100,
                             input_length=26,
                             weights=[loaded_embedding_matrix],
                             trainable=True),
    Bidirectional(LSTM(64, dropout=0.3, return_sequences=True, recurrent_dropout=0.3, kernel_regularizer=l2(0.001))),
    Bidirectional(LSTM(32, dropout=0.3, return_sequences=False, recurrent_dropout=0.3, kernel_regularizer=l2(0.001))),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(3, activation='softmax')
])
opt=tf.keras.optimizers.AdamW(learning_rate=0.001,weight_decay=0.0001)
dep_model6.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])
dep_model6.summary()

In [ ]:
callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True),
           tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-6)]
history6=dep_model6.fit(x_train,y_train,
                        epochs=20,
                        validation_data=(x_test,y_test),
                        batch_size=32,
                        callbacks=callbacks)

In [ ]:
dep_model6.evaluate(x_train,y_train),dep_model5.evaluate(x_test,y_test)

In [ ]:
y_test_classes=np.argmax(y_test,axis=1)
y_pred_classes=np.argmax(dep_model6.predict(x_test),axis=1)
cm=confusion_matrix(y_test_classes,y_pred_classes)
class_labels=le.classes_
plot_confusion_matrix(cm,class_labels)

In [ ]:
tweets=["I didn't like it ",
        "I was a boring movie",
        'What a nice girl',
        'She is a positive girl',
        'fucked up',
        'you suck',
        'I got a great feeling about this',
        'I am beautiful',
        'She is a woman',
        'I am a bastard lol!',
        'Mind your language',
        'I am laughing',
        'You are truely blessed',
        "I love to laugh, it's the best feeling!",
        'He is showing off his abs']
for tweet in tweets:
  predicted_sentiment = predict_sentiment(tweet, preprocessing_pipeline, tokenizer1, dep_model6)
  print(f"Final Predicted Sentiment: {predicted_sentiment}")

In [ ]:
plot_loss_acc_curves(history6)

## Implementing bilstm + attention

In [ ]:
from keras.layers import Attention,Layer,Input
import tensorflow.keras.backend as K
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    def build(self,input_shape):
      self.W=self.add_weight(name='attention_weights',
                             shape=(input_shape[-1],1),
                             initializer='random_normal',
                             trainable=True,
                             )
      self.b=self.add_weight(name='attention_bias',
                             shape=(1,),
                             initializer='zeros',
                             trainable=True)
      super(AttentionLayer,self).build(input_shape)
    def call(self,inputs):
      score=K.tanh(K.dot(inputs,self.W)+self.b)
      attention_weights=K.softmax(score,axis=1)
      context_vector=inputs*attention_weights
      return K.sum(context_vector,axis=1)

In [ ]:
inputs=Input(shape=(26,))
x=Embedding(input_dim=vocab_size,
            output_dim=100,
            weights=[loaded_embedding_matrix],
            trainable=True)(inputs)
x=Bidirectional(LSTM(units=64,return_sequences=True,dropout=0.3,recurrent_dropout=0.3))(x)
x=AttentionLayer()(x)
x=Dropout(0.3)(x)
x=Dense(64,activation='relu')(x)
output=Dense(3,activation='softmax')(x)
dep_model7=tf.keras.models.Model(inputs=inputs,outputs=output)

opt=tf.keras.optimizers.AdamW(learning_rate=0.001,weight_decay=0.0001)
dep_model7.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
dep_model7.summary()

In [ ]:
callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=1,factor=0.5,min_lr=1e-6)
]
dep_model7.fit(x_train,y_train,
               epochs=20,
               validation_data=(x_test,y_test),
               batch_size=32,
               callbacks=callbacks)

In [ ]:
dep_model7.evaluate(x_train,y_train),dep_model7.evaluate(x_test,y_test)

## Using BERT

In [ ]:
!nvcc --version

In [ ]:
from transformers import BertTokenizer,BertForSequenceClassification,Trainer, TrainingArguments,default_data_collator,AutoModelForSequenceClassification,AutoTokenizer,EarlyStoppingCallback
import html
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from datasets import Dataset
from peft import LoraConfig, get_peft_model
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
csv_file = os.path.join(path, "twitter_training.csv")
train_data = pd.read_csv(csv_file,header=None)
csv_file=os.path.join(path,'twitter_validation.csv')
test_data=pd.read_csv(csv_file,header=None)

train_data.head()

In [ ]:
train_data.columns=['tweet_id','state','sentiment','tweet']
test_data.columns=['tweet_id','state','sentiment','tweet']

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
train_data1=train_data[(train_data['sentiment'] != 'Irrelevant')]
np.unique(train_data1['sentiment'])

In [ ]:
def data_preprocessing(tweet):
  if isinstance(tweet,str):
    tweet=html.unescape(tweet)
    tweet=re.sub(r'\s+',' ',tweet).strip()
    tweet=emoji.demojize(tweet,language='en')
  return tweet

In [ ]:
train_data1['tweet_processed']=train_data1['tweet'].apply(data_preprocessing)

In [ ]:
train_data1['tweet_processed']

In [ ]:
x=list(train_data1['tweet_processed'].astype(str))
y=list(train_data1['sentiment'])

In [ ]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(y)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,labels,test_size=0.2,random_state=42)

In [ ]:
train_encodings=tokenizer(x_train,truncation=True,padding=True,return_tensors='pt')
train_encodings

In [ ]:
test_encodings=tokenizer(x_test,truncation=True,padding=True,return_tensors='pt')
test_encodings

In [ ]:
y_train_tensor = torch.tensor(y_train.tolist())
y_test_tensor = torch.tensor(y_test.tolist())
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'],
                                   'attention_mask': train_encodings['attention_mask'],
                                   'labels': y_train_tensor})

test_dataset = Dataset.from_dict({'input_ids': test_encodings['input_ids'],
                                  'attention_mask': test_encodings['attention_mask'],
                                  'labels': y_test_tensor})

In [ ]:
"""dataset = tf.data.Dataset.from_tensor_slices((
    dict(encodings),
    labels
))
batch_size=16
train_dataset=dataset.shuffle(len(x)).batch(batch_size)"""

In [ ]:
"""from torch.utils.data import Dataset
import torch

class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])

        return item

    def __len__(self):
        return len(self.labels)"""


In [ ]:
"""train_dataset=TweetDataset(train_encodings,labels)"""

In [ ]:
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)
lora_config=LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['query','key','value'],
    lora_dropout=0.1,
    bias='none',
    task_type='SEQ_CLS'
)

model=get_peft_model(model,lora_config)
model.print_trainable_parameters()

In [ ]:
class CustomTrainer(Trainer):
    def get_train_dataloader(self,train_dataset=None):
        return train_dataloader

    def get_eval_dataloader(self,eval_dataset=None):
        return eval_dataloader

In [ ]:
def compute_metrics(eval_pred):
      logits, labels = eval_pred
      preds = np.argmax(logits, axis=1)
      acc = accuracy_score(labels, preds)
      return {"accuracy": acc}

In [ ]:
def predict_sentiment_hf(tweet_text, tokenizer, model):
    inputs = tokenizer(tweet_text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items() if k != "token_type_ids"}  # Remove token_type_ids for DistilBERT
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()

    labels = ["Negative", "Neutral", "Positive"]
    predicted = labels[np.argmax(probs)]

    print(f"Tweet: {tweet_text}")
    print(f"Probabilities: {probs}")
    print(f"Predicted Sentiment: {predicted}")

In [ ]:
train_dataloader = DataLoader(
    train_dataset, batch_size=8, num_workers=2, collate_fn=default_data_collator
)

eval_dataloader = DataLoader(
    test_dataset, batch_size=8, num_workers=2, collate_fn=default_data_collator
)


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    warmup_steps=100,
    num_train_epochs=5,
    logging_dir="./logs",
    learning_rate=2e-5,
    report_to="none",
    fp16=True,
    logging_strategy='epoch',
    eval_strategy='epoch',
    save_strategy='epoch',
    dataloader_num_workers=2,

)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.save_model("/content/MyDrive/MyDrive/twitter_sentiment_analysis/bert_model")
tokenizer.save_pretrained("/content/MyDrive/MyDrive/twitter_sentiment_analysis/bert_tokenizer")

In [ ]:
loaded_tokenizer = BertTokenizer.from_pretrained('/content/MyDrive/MyDrive/twitter_sentiment_analysis/bert_tokenizer')

loaded_model = BertForSequenceClassification.from_pretrained(
    '/content/MyDrive/MyDrive/twitter_sentiment_analysis/bert_model',
    num_labels=3,
    ignore_mismatched_sizes=True
)

In [ ]:
results=trainer.evaluate()
results

In [ ]:
import numpy as np
tweets=["I didn't like it ",
        "It was a boring movie",
        'What a nice girl',
        'She is a positive girl',
        'fucked up',
        'you suck',
        'I got a great feeling about this',
        'I am beautiful',
        'She is a woman',
        'I am a bastard lol!',
        'Mind your language',
        'That was hillerious',
        'I am laughing',
        'I am going to kerala today',
        'You are truely blessed',
        "I love to laugh, it's the best feeling!",
        'He is showing off his abs',
        "what I'm saying is that you are literally the Monday of my life.",
        "Life's good, you should get one."]
for tweet in tweets:
  predict_sentiment_hf(tweet, loaded_tokenizer, loaded_model)
  print('\n')

In [ ]:
model.eval()
preds = []
true_labels = []

with torch.no_grad():
    for batch in eval_dataloader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, preds)
report = classification_report(true_labels, preds)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


In [ ]:
def predict_sentiment_hf(tweet_text, tokenizer, model):
    inputs = tokenizer(tweet_text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()

    labels = ["Negative", "Neutral", "Positive"]
    predicted = labels[np.argmax(probs)]

    print(f"Tweet: {tweet_text}")
    print(f"Probabilities: {probs}")
    print(f"Predicted Sentiment: {predicted}")

In [ ]:
import tweepy

api_key = '<api-key>'
api_key_secret = '<api-secret-key>'
bearer_token = '<bearer_token>'

client = tweepy.Client(bearer_token=bearer_token)

query = 'AI OR machine learning -is:retweet lang:en'
tweets = client.search_recent_tweets(
    query=query,
    max_results=10,
    tweet_fields=["lang"]
)

In [ ]:
for tweet in tweets.data:
    if tweet.lang == "en":
        print(tweet.text)

In [ ]:
for tweet in tweets.data:
    if tweet.lang == "en":
        text = tweet.text
        predict_sentiment_hf(text, tokenizer, model)
        print('\n')

## DistilBert

In [ ]:
tokenizer1 = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model1 = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

In [ ]:
train_encodings=tokenizer1(x_train,truncation=True,padding=True,return_tensors='pt')
test_encodings=tokenizer1(x_test,truncation=True,padding=True,return_tensors='pt')
train_encodings

In [ ]:
y_train_tensor = torch.tensor(y_train.tolist())
y_test_tensor = torch.tensor(y_test.tolist())

train_dataset=Dataset.from_dict({'input_ids':train_encodings['input_ids'],
                                 'attention_mask':train_encodings['attention_mask'],
                                 'labels':y_train_tensor})
test_dataset=Dataset.from_dict({'input_ids':test_encodings['input_ids'],
                                'attention_mask':test_encodings['attention_mask'],
                                'labels':y_test_tensor})

In [ ]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules = ['q_lin', 'k_lin', 'v_lin'],
    lora_dropout=0.1,
    bias='none',
    task_type='SEQ_CLS'
)

model=get_peft_model(model1,lora_config)
model.print_trainable_parameters()

In [ ]:
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))
print("TF32 supported:", torch.backends.cuda.matmul.allow_tf32)

In [ ]:
train_dataloader = DataLoader(
    train_dataset, batch_size=8, num_workers=2, collate_fn=default_data_collator
)

eval_dataloader = DataLoader(
    test_dataset, batch_size=8, num_workers=2, collate_fn=default_data_collator
)


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    warmup_steps=100,
    num_train_epochs=10,
    logging_dir="./logs",
    learning_rate=2e-5,
    report_to="none",
    fp16=True,
    logging_strategy='epoch',
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    dataloader_num_workers=2,
)

trainer = CustomTrainer(
    model=model1,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

In [ ]:
results=trainer.evaluate()
results

In [ ]:
train_metrics = trainer.evaluate(eval_dataset=train_dataset)
print("Train:", train_metrics)

val_metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Validation:", val_metrics)


In [ ]:
trainer.save_model("/content/MyDrive/MyDrive/twitter_sentiment_analysis/distilbert_model")
tokenizer.save_pretrained("/content/MyDrive/MyDrive/twitter_sentiment_analysis/distilbert_tokenizer")

In [ ]:
loaded_tokenizer1=AutoTokenizer.from_pretrained('/content/MyDrive/MyDrive/twitter_sentiment_analysis/distilbert_tokenizer')
loaded_model1=AutoModelForSequenceClassification.from_pretrained('/content/MyDrive/MyDrive/twitter_sentiment_analysis/distilbert_model',num_labels=3)

In [ ]:
def predict_sentiment_hf(tweet_text, tokenizer, model):
    inputs = tokenizer(tweet_text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items() if k != "token_type_ids"}  # Remove token_type_ids for DistilBERT
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()

    labels = ["Negative", "Neutral", "Positive"]
    predicted = labels[np.argmax(probs)]

    print(f"Tweet: {tweet_text}")
    print(f"Probabilities: {probs}")
    print(f"Predicted Sentiment: {predicted}")


In [ ]:
tweets=["I didn't like it ",
        "I was a boring movie",
        'What a nice girl',
        'She is a positive girl',
        'fucked up',
        'you suck',
        'I got a great feeling about this',
        'I am beautiful',
        'She is a woman',
        'I am a bastard lol!',
        'Mind your language',
        'That was hillerious',
        'I am laughing',
        'I am going to kerala today',
        'You are truely blessed',
        "I love to laugh, it's the best feeling!",
        'He is showing off his abs',
        "what I'm saying is that you are literally the Monday of my life.",
        "Life's good, you should get one."]
for tweet in tweets:
  predict_sentiment_hf(tweet, tokenizer1, model1)
  print('\n')

In [ ]:
for tweet in tweets.data:
    if tweet.lang == "en":
        text = tweet.text
        predict_sentiment_hf(text, tokenizer, model1)
        print('\n')

## RoBERTa

In [ ]:
tokenizer2=AutoTokenizer.from_pretrained('roberta-base')
model2=AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=3)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,labels,test_size=0.2,random_state=42)

In [ ]:
train_tokenizer2=tokenizer2(x_train,truncation=True,padding=True,return_tensors='pt')
test_tokenizer2=tokenizer2(x_test,truncation=True,padding=True,return_tensors='pt')

In [ ]:
train_dataset=Dataset.from_dict({'input_ids':train_tokenizer2['input_ids'],
                                 'attention_mask':train_tokenizer2['attention_mask'],
                                 'label':y_train_tensor})
test_dataset=Dataset.from_dict({'input_ids':test_tokenizer2['input_ids'],
                                'attention_mask':test_tokenizer2['attention_mask'],
                                'label':y_test_tensor})

In [ ]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules = ['query', 'key', 'value'],
    lora_dropout=0.1,
    bias='none',
    task_type='SEQ_CLS'
)

model2=get_peft_model(model2,lora_config)
model2.print_trainable_parameters()

In [ ]:
train_dataloader = DataLoader(
    train_dataset, batch_size=8, num_workers=2, collate_fn=default_data_collator
)

eval_dataloader = DataLoader(
    test_dataset, batch_size=8, num_workers=2, collate_fn=default_data_collator
)


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    warmup_steps=100,
    num_train_epochs=10,
    logging_dir="./logs",
    learning_rate=2e-5,
    report_to="none",
    fp16=True,
    logging_strategy='epoch',
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    dataloader_num_workers=2,
)

trainer = CustomTrainer(
    model=model2,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

In [ ]:
trainer.save_model("/content/MyDrive/MyDrive/twitter_sentiment_analysis/roberta_model")
tokenizer2.save_pretrained("/content/MyDrive/MyDrive/twitter_sentiment_analysis/roberta_tokenizer")

In [ ]:
results = trainer.evaluate(eval_dataset=test_dataset)
print(results)

In [ ]:
loaded_tokenizer=AutoTokenizer.from_pretrained('/content/MyDrive/MyDrive/twitter_sentiment_analysis/roberta_tokenizer')
loaded_model=AutoModelForSequenceClassification.from_pretrained('/content/MyDrive/MyDrive/twitter_sentiment_analysis/roberta_model',num_labels=3)

In [ ]:
def predict_sentiment_roberta(tweet_text, tokenizer, model):
    inputs = tokenizer(tweet_text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()

    labels = ["Negative", "Neutral", "Positive"]
    predicted = labels[np.argmax(probs)]

    print(f"Tweet: {tweet_text}")
    print(f"Probabilities: {probs}")
    print(f"Predicted Sentiment: {predicted}")

In [ ]:
tweets=["I didn't like it ",
        "I was a boring movie",
        'What a nice girl',
        'She is a positive girl',
        'fucked up',
        'you suck',
        'I got a great feeling about this',
        'I am beautiful',
        'She is a woman',
        'I am a bastard lol!',
        'Mind your language',
        'That was hillerious',
        'I am laughing',
        'I am going to kerala today',
        'You are truely blessed',
        "I love to laugh, it's the best feeling!",
        'He is showing off his abs',
        "what I'm saying is that you are literally the Monday of my life.",
        "Life's good, you should get one."]
for tweet in tweets:
  predict_sentiment_roberta(tweet, loaded_tokenizer, loaded_model)
  print('\n')

In [ ]:
import tweepy

api_key = '<api-key>'
api_key_secret = '<api-secret-key>'
bearer_token = '<bearer_token>'

client = tweepy.Client(bearer_token=bearer_token)

query = 'AI OR machine learning -is:retweet lang:en'
tweets = client.search_recent_tweets(
    query=query,
    max_results=10,
    tweet_fields=["lang"]
)

In [ ]:
for tweet in tweets.data:
    if tweet.lang == "en":
        text = tweet.text
        predict_sentiment_roberta(text, loaded_tokenizer, loaded_model)
        print('\n')